In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import gensim
import gensim.models.keyedvectors as word2vec

In [2]:
#load the models:
fname = "C:/Users/Suryansh/Desktop/ucd/Stage4/FYP/New/Semester2/Word2vec_visualise/Models/bias"

In [3]:
model = gensim.models.keyedvectors.KeyedVectors.load(fname)

In [4]:
max_size = len(model.wv.vocab)-1
w2v = np.zeros((max_size,model.layer1_size))
w2v

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
african_dict = ['african','africans','black','blacks','chief','chiefs','coloured','kaffir','kaffer','cafri','locals','macaca','macaque','mulatto','mulattoes','native','negress','negro','negroes','nigger','pickaninny','pygmy','pygmies','savage','savages','slave','slaves','ewe','dogon','dinka','danakil','adali','odali','abyssinians','asante','berbers','amazighs','maasai','pahouin','san','somalis','somali','sudanese','sudan','gambian','gambia','algeria','algerian','angola','angolan','benin','beninese','camerron','cameroonian','chad','chadian','comoros','comoran','egypt','egyptian','eritrea','eritrean','ethiopia','ethiopian','gabon','gabonese','ghana','ghanian','liberia','liberian','libya','libyan','madagascar','madagascan','malawi','malawian','mali','malian','mauritania','mauretanian','mauritius','mauritian','morocco','moroccan','mozambique','mozambiquean','niger','nigerian','senegal','senegalese','seychelles','seychellois','swaziland','swazi','togo','togolese','tunisia','tunisian','uganda','ugandan']

In [12]:
ind_dict = ['india','indian','indians','assamese','bengali','bhil','dhivehi','dogra','garhwali','gujarati','kutchi','hindavi','awadhi','bhojpuri','rajasthanis','marwaris','magahi','muhajirs','magahi','nagpuri','kashmiri','khas','konkani','kumaoni','maithils','marathi','odia','punjabi','pahari','parsi','rohingya','sindhi','memons','saraiki','saurashtra','sinhalese','sylheti','tharu']

In [7]:
brit_dict = ["brit","brits","british","britisher","britishers","english","englishman","englishmen","european","europeans","white","whites","american","americans","scotch","scotchman","irish","irishman","german","germans"]

In [8]:
crim_dict = ["criminal","criminals","offense","offender","lewd","fraud","abuse","assailant","assault","bribe","bribery","burglar","burglary","cheat","convict","conviction","corrupt","corruption","danger","dangerous","illegal","disobediance","immoral","imprisoned","incarceration","inmate","theft","thief","rape","robber","robbery","threatening","vagrant","vagrancy"]

In [10]:
smrt_dict = ["smart","smarts","smarter","wit","wise","wisdom","intellect","intelligent","intelligence","brilliant","brilliance","brains","clever","cleverness"]

In [11]:
dumb_dict = ["simpleness","simplicity","simpleton","dumb","dumbness","fool","foolish","foolishness","mindless","mindlessness","senseless","senselessness","stupid","stupidity","stupidness"]

In [13]:
with open("metadata.tsv","w+") as file_metadata:
    file_metadata.write("Word\tCategory\n")
    for i,word in enumerate(model.wv.index2word[:max_size]):
        w2v[i] = model.wv[word]
        if(word in african_dict):
            file_metadata.write("%s\t%s\n" % (word,'terms_african'))
        elif(word in ind_dict):
            file_metadata.write("%s\t%s\n" % (word,'terms_indian'))
        elif(word in brit_dict):
            file_metadata.write("%s\t%s\n" % (word,'terms_caucasian'))
        elif(word in crim_dict):
            file_metadata.write("%s\t%s\n" % (word,'terms_criminality'))
        elif(word in smrt_dict):
            file_metadata.write("%s\t%s\n" % (word,'terms_smart'))
        elif(word in dumb_dict):
            file_metadata.write("%s\t%s\n" % (word,'terms_dumb'))
        else:
            file_metadata.write("%s\t%s\n" % (word,'none'))

In [14]:
w2v

array([[-2.03846812e+00,  2.50827193e+00,  4.16977048e-01, ...,
        -3.06430817e+00, -1.03770852e+00,  1.49504697e+00],
       [-8.28388214e-01,  5.27804613e-01,  3.30885410e+00, ...,
        -1.12653464e-01,  3.92413974e+00,  1.20077990e-01],
       [ 3.60598505e-01,  1.36050177e+00, -2.08310574e-01, ...,
        -3.76490206e-01, -5.32922924e-01,  1.65610766e+00],
       ...,
       [-2.48545338e-03,  3.17125395e-03, -3.72075569e-03, ...,
         3.82435531e-03, -7.62992480e-04,  4.36963513e-03],
       [-4.64878278e-03,  3.92424222e-03,  4.34570434e-03, ...,
         2.11275392e-03,  2.60445382e-03,  1.95318297e-03],
       [-3.21174995e-03, -2.71514873e-03,  4.67777951e-03, ...,
        -1.40409265e-03,  7.27262450e-05,  5.06083306e-04]])

In [15]:
sess = tf.InteractiveSession()

In [16]:
#Let us create a 2D tensor called embedding that holds our embeddings.
with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')

In [17]:
tf.global_variables_initializer().run()

In [18]:
path = 'tensorboard'

In [19]:
# let us create an object to Saver class which is actually used to 
#save and restore variables to and from our checkpoints
saver = tf.train.Saver()

In [20]:
# using file writer, we can save our summaries and events to our event file.
writer = tf.summary.FileWriter(path, sess.graph)

In [21]:
# adding into projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

In [22]:
# Specify the width and height of a single thumbnail.
projector.visualize_embeddings(writer, config)

saver.save(sess, path+'/model.ckpt', global_step=max_size)

'tensorboard/model.ckpt-788791'